In [1]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

In [2]:
import pickle

with open('URM_train_validation.pkl', 'rb') as f:
    URM_train_validation = pickle.load(f)
with open('URM_test.pkl', 'rb') as f:
    URM_test = pickle.load(f)

In [3]:
from Hybrid import *
from Recommenders.SLIM.SLIMElasticNetRecommender import *
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
## SLIM
slim = SLIMElasticNetRecommender(URM_train_validation)
slim.fit(topK=8894, l1_ratio=0.05565733019999427, alpha=0.0012979360257937668)#, workers=7)
## RP3BETA
rp3beta = RP3betaRecommender(URM_train_validation)
rp3beta.fit(alpha=0.20026352123406477, beta=0.15999879728761354, topK=32)
## USER KNN
userknn = UserKNNCFRecommender(URM_train_validation)
userknn.fit(topK=469, shrink=38, similarity='asymmetric', normalize=True,
                                       feature_weighting='TF-IDF', asymmetric_alpha=0.40077406933762383)
## ITEM KNN
itemknn = ItemKNNCFRecommender(URM_train_validation)
itemknn.fit(topK=31, shrink=435, similarity='tversky', normalize=True,
                                       feature_weighting='BM25', tversky_alpha=0.17113169506422393, tversky_beta=0.5684024974085575)
## EASE_R
ease_r = EASE_R_Recommender(URM_train_validation)
ease_r.fit(topK=24, l2_norm=37.54323189430143)

model = LinearHybridRecommender

from Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[10])

SLIMElasticNetRecommender: URM Detected 205 ( 1.6%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 109 ( 0.5%) items with no interactions.
SLIMElasticNetRecommender: Processed 22222 (100.0%) in 4.51 min. Items per second: 82.19
RP3betaRecommender: URM Detected 205 ( 1.6%) users with no interactions.
RP3betaRecommender: URM Detected 109 ( 0.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 7272.35 column/sec. Elapsed time 3.06 sec
UserKNNCFRecommender: URM Detected 205 ( 1.6%) users with no interactions.
UserKNNCFRecommender: URM Detected 109 ( 0.5%) items with no interactions.
Similarity column 12638 (100.0%), 10232.89 column/sec. Elapsed time 1.24 sec
ItemKNNCFRecommender: URM Detected 205 ( 1.6%) users with no interactions.
ItemKNNCFRecommender: URM Detected 109 ( 0.5%) items with no interactions.
Similarity column 22222 (100.0%), 10202.35 column/sec. Elapsed time 2.18 sec
EASE_R_Recommender: URM Detected 205 ( 1.6%) users with

In [4]:
knn = TwoScoresHybridRecommender(URM_train_validation, userknn, itemknn)
knn.fit(alpha=0.022195783788315104)

TwoScoresHybridRecommender: URM Detected 205 ( 1.6%) users with no interactions.
TwoScoresHybridRecommender: URM Detected 109 ( 0.5%) items with no interactions.


In [5]:
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender

ials = IALSRecommender(URM_train_validation)
ials.load_model(folder_path="slim_models/", file_name="IALS_train_validation")

IALSRecommender: URM Detected 205 ( 1.6%) users with no interactions.
IALSRecommender: URM Detected 109 ( 0.5%) items with no interactions.
IALSRecommender: Loading model from file 'slim_models/IALS_train_validation'
IALSRecommender: Loading complete


In [19]:
hybrid = LinearHybridRecommender(URM_train_validation, [slim, rp3beta, knn, ease_r])
hybrid.fit(alphas=[0.8857667100747117, 0.6379807942443021, 0.0690929184825888, 0.11953478623354052])
hybrid_recommender = TwoScoresHybridRecommender(URM_train_validation, hybrid, ials)

SLIMElasticNetRP3betaTwoScoresHybridEASE_R_HybridRecommender: URM Detected 205 ( 1.6%) users with no interactions.
SLIMElasticNetRP3betaTwoScoresHybridEASE_R_HybridRecommender: URM Detected 109 ( 0.5%) items with no interactions.
TwoScoresHybridRecommender: URM Detected 205 ( 1.6%) users with no interactions.
TwoScoresHybridRecommender: URM Detected 109 ( 0.5%) items with no interactions.


In [20]:
import optuna as op
def objective(trial):
    alpha = trial.suggest_float('alpha', 0, 1)
    
    # Create hybrid recommender with the suggested weights
    hybrid_recommender.fit(alpha=alpha)
    
    # Evaluate the hybrid recommender
    result, _ = evaluator_validation.evaluateRecommender(hybrid_recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [21]:
best_params = {
    "alpha": 1,
}
study_name = "hybrid-IALS-study"  # Unique identifier of the study.
storage_name = "sqlite:///db.db"
#op.delete_study(study_name, storage_name)
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize", load_if_exists=True)
#study.enqueue_trial(best_params)
study.optimize(objective, n_trials=100)

/var/folders/l3/n3yjvx_d2g9dy0ztkyft2th40000gn/T/ipykernel_19066/1644483355.py:6: FutureWarning: delete_study(): Please give all values as keyword arguments. See https://github.com/optuna/optuna/issues/3324 for details.
  op.delete_study(study_name, storage_name)
[I 2023-12-29 00:53:07,090] A new study created in RDB with name: hybrid-IALS-study


EvaluatorHoldout: Processed 10435 (100.0%) in 9.54 sec. Users per second: 1094


[I 2023-12-29 00:53:16,657] Trial 0 finished with value: 0.052030673633453296 and parameters: {'alpha': 1.0}. Best is trial 0 with value: 0.052030673633453296.


EvaluatorHoldout: Processed 10435 (100.0%) in 9.60 sec. Users per second: 1087


[I 2023-12-29 00:53:26,278] Trial 1 finished with value: 0.036898932165104155 and parameters: {'alpha': 0.04804232856077917}. Best is trial 0 with value: 0.052030673633453296.


...

[I 2023-12-29 01:08:15,687] Trial 97 finished with value: 0.05198024429385303 and parameters: {'alpha': 0.9572803593406405}. Best is trial 63 with value: 0.05204516622173605.


EvaluatorHoldout: Processed 10435 (100.0%) in 9.90 sec. Users per second: 1054


[I 2023-12-29 01:08:25,610] Trial 98 finished with value: 0.05175163711867185 and parameters: {'alpha': 0.8552584517022478}. Best is trial 63 with value: 0.05204516622173605.


EvaluatorHoldout: Processed 10435 (100.0%) in 10.25 sec. Users per second: 1018


[I 2023-12-29 01:08:35,889] Trial 99 finished with value: 0.05173920566469749 and parameters: {'alpha': 0.8904871843260057}. Best is trial 63 with value: 0.05204516622173605.


In [16]:
study.best_params

{'alpha': 0.8857667100747117,
 'beta': 0.6379807942443021,
 'gamma': 0.0690929184825888,
 'delta': 0.11953478623354052}